In [ ]:
import pandas as pd
import json
import numpy as np


used_classes = [('Airliner', 'n02690373'),
                ('Sorrel', 'n02389026'),
                ('Jack-o’-lantern', 'n03590841'),
                ('Panda', 'n02510455'),
                ('Anemone fish', 'n02607072')]

# CHANGE THIS
# Provide filenames of written EEG recordings
eeg_files_pref = ['a_1', 'a_2', 
                  'v_1', 'v_2', 
                  'm_1', 'm_2']

eeg_data_dir = 'eeg/'
eeg_file_name = eeg_files_pref[5]
eeg_file_path = eeg_data_dir + eeg_file_name + '.csv'

eeg_data = pd.read_csv(eeg_file_path, 
                       names=['counter', 'interp', 'F3', 'FC5', 'AF3', 'F7', 'T7', 'P7', 'O1', 
                              'O2', 'P8', 'T8', 'F8', 'AF4', 'FC6', 'F4', 'RAW_CQ', 'GYROX', 'ts'])

eeg_data['class'] = np.nan
eeg_data['img_num'] = np.nan

image_timestamps_path = eeg_data_dir + eeg_file_name + '_timestamps.json'
with open(image_timestamps_path) as f:
    image_timestamps = json.load(f)

print(eeg_file_path)
print(image_timestamps_path)
print(len(image_timestamps))

In [ ]:
eeg_data = eeg_data.drop(['interp', 'RAW_CQ', 'GYROX'], axis=1)

eeg_data.head()

In [ ]:
img_relation_delay_ns = 40 * (10 ** 6)
viewed_classes = [name for name, n in classes]

for i in range(len(image_timestamps) - 1):
    cl_num = i // 71
    img_num = i % 71 - 1
    if img_num != -1:
        eeg_data.loc[(eeg_data['ts'] >  image_timestamps[i] + img_relation_delay_ns) 
                     & (eeg_data['ts'] <  image_timestamps[i + 1]), 'class'] = viewed_classes[cl_num]
        eeg_data.loc[(eeg_data['ts'] >  image_timestamps[i] + img_relation_delay_ns) 
                     & (eeg_data['ts'] <  image_timestamps[i + 1]), 'img_num'] = img_num
        
        
eeg_data = eeg_data.dropna()
eeg_data['img_num'] = eeg_data['img_num'].astype(int)
eeg_data.head()

In [ ]:
import os.path


eeg_processed_dir = 'eeg_processed/'
if not os.path.isdir(eeg_processed_dir):
    print("Creating " + eeg_processed_dir)
    os.makedirs(eeg_processed_dir)

for i, cl in enumerate(viewed_classes):
    print(i, cl)
    for j in range(70):
        img_eeg = eeg_data.loc[(eeg_data['class'] == cl) & (eeg_data['img_num'] == j)]
        img_eeg = img_eeg.reset_index(drop=True)
        img_eeg_path = eeg_processed_dir + '{}_{}_{}.csv'.format(eeg_file_name[0] + eeg_file_name[-1], i, j)
        if os.path.exists(img_eeg_path):
            print ("File already exist: " + img_eeg_path)
        else:
            img_eeg.to_csv(img_eeg_path)